## Train a neural network to check if there is any function mapping from a subset of bits to the remainder

I tried finding a way to calculate the modulo 10 of a number just by its last 4 bits, but that doesn't work.

In [4]:
bin(10)

'0b1010'

In [89]:
def BitstringToTensor(string, dtype):
    return tensor([int(c) for c in string], dtype=dtype)

In [92]:
def LastNBits(x, n):
    "returns the last n bits of x as a tensor"
    return str.zfill(bin(x)[2:][-n:], n)

In [96]:
def LastNBitsToTensor(x, n):
    return BitstringToTensor(str.zfill(bin(x)[2:][-n:], n), dtype=torch.float32)

In [98]:
def Sample(x, InputSize):
    Input  = LastNBitsToTensor(x, InputSize)
    Output = LastNBitsToTensor(x%10, 4)
    return Input, Output

In [207]:
def prepend(v, T):
    return torch.cat((tensor([v]), T))

In [226]:
def SampleB(x, InputSize):
    Input  = prepend(x.bit_length(), LastNBitsToTensor(x, InputSize))
    Output = LastNBitsToTensor(x%10, 4)
    return Input, Output

In [224]:
def random_range(start, stop):
    R = list(range(start, stop))
    random.shuffle(R)
    yield from R

In [261]:
Sample(14, 4)

(tensor([1., 1., 1., 0.]), tensor([0., 1., 0., 0.]))

In [97]:
LastNBitsToTensor(20, 5)

tensor([1., 0., 1., 0., 0.])

In [93]:
LastNBits(20, 5)

'10100'

In [29]:
str.zfill(bin(2)[2:], 4)

'0010'

In [110]:
for i in range(100):
    if i%10 == 0: print('--------------------')
    print(i%10, LastNBits(i, 4), LastNBits(i%10, 4))

--------------------
0 0000 0000
1 0001 0001
2 0010 0010
3 0011 0011
4 0100 0100
5 0101 0101
6 0110 0110
7 0111 0111
8 1000 1000
9 1001 1001
--------------------
0 1010 0000
1 1011 0001
2 1100 0010
3 1101 0011
4 1110 0100
5 1111 0101
6 0000 0110
7 0001 0111
8 0010 1000
9 0011 1001
--------------------
0 0100 0000
1 0101 0001
2 0110 0010
3 0111 0011
4 1000 0100
5 1001 0101
6 1010 0110
7 1011 0111
8 1100 1000
9 1101 1001
--------------------
0 1110 0000
1 1111 0001
2 0000 0010
3 0001 0011
4 0010 0100
5 0011 0101
6 0100 0110
7 0101 0111
8 0110 1000
9 0111 1001
--------------------
0 1000 0000
1 1001 0001
2 1010 0010
3 1011 0011
4 1100 0100
5 1101 0101
6 1110 0110
7 1111 0111
8 0000 1000
9 0001 1001
--------------------
0 0010 0000
1 0011 0001
2 0100 0010
3 0101 0011
4 0110 0100
5 0111 0101
6 1000 0110
7 1001 0111
8 1010 1000
9 1011 1001
--------------------
0 1100 0000
1 1101 0001
2 1110 0010
3 1111 0011
4 0000 0100
5 0001 0101
6 0010 0110
7 0011 0111
8 0100 1000
9 0101 1001
-------------

### Train the neural network

In [13]:
from fastai.vision.all import *

In [17]:
def dense_network(input_size, hidden_size, output_size, num_layers, dropout_p):
    assert num_layers >= 2, 'need at least two layers'
    layers = [nn.Linear(input_size, hidden_size),
              nn.Dropout(p=dropout_p),
              nn.ReLU()]
    for _ in range(num_layers - 2):
        layers.append(nn.Linear(hidden_size, hidden_size))
        layers.append(nn.Dropout(p=dropout_p))
        layers.append(nn.ReLU())
    return nn.Sequential(*layers,
                         nn.Linear(hidden_size, output_size),
                         nn.Sigmoid())

In [259]:
model = dense_network(5, 50, 4, 10, 0.1)

In [260]:
# Define the loss function and optimizer
criterion = nn.MSELoss()  # You can choose an appropriate loss function
optimizer = Adam(model.parameters(), lr=0.001)  # You can adjust the learning rate

In [276]:
# Training loop
num_epochs = 1000  # Adjust the number of epochs as needed
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i in random_range(0, 100):
        inputs, targets = SampleB(i, 4)
        # inputs = torch.cat((tensor([i.bit_count()]), inputs))
        
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        
        # Compute the loss
        loss = criterion(outputs, targets)
        
        # Backpropagation
        loss.backward()
        
        # Update the weights
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
    if epoch % 50 == 49:  # Print every 10 mini-batches
    # if 1:
        print(f"Epoch {epoch + 1}, Loss: {running_loss / 50}")
        running_loss = 0.0

print("Training complete")

Epoch 50, Loss: 0.32490048557754286
Epoch 100, Loss: 0.32534436663281796
Epoch 150, Loss: 0.3211137062347687
Epoch 200, Loss: 0.3212417855858803
Epoch 250, Loss: 0.34048370730131866
Epoch 300, Loss: 0.32041027083992957
Epoch 350, Loss: 0.31957923568503704
Epoch 400, Loss: 0.32067089034011587
Epoch 450, Loss: 0.31827987428755455
Epoch 500, Loss: 0.329678179913717
Epoch 550, Loss: 0.321901417375718
Epoch 600, Loss: 0.3233600011654198
Epoch 650, Loss: 0.32718296147117143
Epoch 700, Loss: 0.322390413103567
Epoch 750, Loss: 0.3128943892851248
Epoch 800, Loss: 0.3137536603220237
Epoch 850, Loss: 0.3178575890118418
Epoch 900, Loss: 0.3162860677218856
Epoch 950, Loss: 0.31627465046770686
Epoch 1000, Loss: 0.3170476002245185
Training complete


In [286]:
def ToBits(x):
    return x
    return torch.heaviside(x - 0.5, Tensor([0]))

In [306]:
with torch.no_grad():
    Input, Target = SampleB(18, 4)
    print(ToBits(model(Input)), Target)

tensor([0.2067, 0.3970, 0.4157, 0.0030]) tensor([1., 0., 0., 0.])


## Difference between divisions

Testing if we can use the remainder of a division with a large multiple of 10 to speed up the computation of modulo 10.

NOTE: There seems to be something fishy going on with the last digit, depending on which exponent we choose. All exponents of two ending in 0 seem to be ending in 6 once calculated. All with 1 end with 2, all with 2 end with 4 and so on. Check this out later!

In [11]:
N = 1<<1001

In [12]:
one = N % 10**1

In [13]:
two = N % 10**2

In [14]:
three = N % 10**3

In [15]:
four = N % 10**4

In [16]:
five = N % 10**5

In [17]:
six = N % 10**6

In [18]:
seven = N % 10**7

In [19]:
eight = N % 10**8

In [20]:
one, two, three, four, five, six, seven, eight

(2, 52, 752, 8752, 38752, 138752, 6138752, 36138752)

In [122]:
N

42860344287450692837937001962400072422456192468221344297750015534814042044997444899727935152627834325103786916702125873007485811427692561743938310298794299215738271099296923941684298420249484567511816728612185899934327765069595070236662175784308251658284785910746168670641719326610497547348822672277504

In [123]:
N % 10**399

42860344287450692837937001962400072422456192468221344297750015534814042044997444899727935152627834325103786916702125873007485811427692561743938310298794299215738271099296923941684298420249484567511816728612185899934327765069595070236662175784308251658284785910746168670641719326610497547348822672277504

It seems like any time we do a `n mod 10**k`, we're really grabbing about the last k decimal digits of the number n. So if we then do a mod 10 afterwards on that value, we will get the result of n mod 10, without having to do the whole computation again.

In [208]:
from line_profiler import LineProfiler

In [209]:
profiler = LineProfiler()
@profiler
def count_sixes(N, D, Cache):
    SixCounter    = 0
    SixCounterMax = 0
    j = 0
    while N > 0:
        # TODO: use continued division for this as well? Or can we just use the remainder instead of the number?
        Digit = N % 10
        if Digit == 6:
            SixCounter   += 1
            SixCounterMax = max(SixCounterMax, SixCounter)
        else:
            SixCounter = 0
        try:
            Q, R = Cache[j]
            R    = (R << 1) | (N & 1)
            if R >= D:
                R = R - D
                N = (Q << 1) | 1
            else:
                N = (Q << 1)
        except KeyError:
            R = N  % D
            N = N // D
        Cache[j] = N, R
        j += 1
    return SixCounterMax

In [213]:
profiler = LineProfiler()
@profiler
def count_sixes(N, D, Cache):
    SixCounter    = 0
    SixCounterMax = 0
    j = 0
    while N > 0:
        try:
            Q, R = Cache[j]
            R    = (R << 1) | (N & 1)
            if R >= D:
                R = R - D
                N = (Q << 1) | 1
            else:
                N = (Q << 1)
        except KeyError:
            R = N  % D
            N = N // D
        Cache[j] = N, R
        j += 1
        
        Digit = R % 10
        if Digit == 6:
            SixCounter   += 1
            SixCounterMax = max(SixCounterMax, SixCounter)
        else:
            SixCounter = 0
        
    return SixCounterMax

In [214]:
%%time
i        = 100_000
Number   = 1<<(i - 1)
print('Building Cache...')
Divisor = 10**399
Cache   = {}
count_sixes(Number, Divisor, Cache)
Number = Number << 1
print('Done!')

Building Cache...
Done!
CPU times: total: 31.2 ms
Wall time: 18 ms


In [211]:
%%time
output = []
for _ in range(1000):
    output.append(count_sixes(Number, Divisor, Cache))
    i      += 1
    Number  = Number<<1
print(output)

[2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 3, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 6, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 3, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 3, 1, 1, 3, 2, 1, 2, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 1, 2, 3, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 3, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 4, 1, 1, 3, 1, 2, 1, 1, 1, 1, 3, 2, 1, 2, 2, 

In [215]:
%%time
output = []
for _ in range(1000):
    output.append(count_sixes(Number, Divisor, Cache))
    i      += 1
    Number  = Number<<1
print(output)

[2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 3, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 6, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 3, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 3, 1, 1, 3, 2, 1, 2, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 1, 2, 3, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 3, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 4, 1, 1, 3, 1, 2, 1, 1, 1, 1, 3, 2, 1, 2, 2, 

In [207]:
[2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 3, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 6, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 3, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 3, 1, 1, 3, 2, 1, 2, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 1, 2, 3, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 3, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 4, 1, 1, 3, 1, 2, 1, 1, 1, 1, 3, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 3, 2, 3, 4, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 3, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 3, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 3, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 3, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 3, 2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 3, 1, 1, 2, 1, 1, 1, 2, 4, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 3, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 3, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 3, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 3, 1, 2, 1, 1, 1, 4, 3, 1, 1, 2, 2, 1, 3, 1, 1, 1, 1, 2, 3, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 3, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 3, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 4, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 3, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 3, 2, 3, 1, 1, 1, 1, 2, 3, 1, 3, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 3, 1, 2, 2, 2, 3, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 3, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 3, 1, 1, 1, 2, 3, 1, 1, 1, 3, 3, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 3, 1, 2, 2, 2, 3, 1, 3, 1, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 3, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2] == [2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 3, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 6, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 3, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 3, 1, 1, 3, 2, 1, 2, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 1, 2, 3, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 3, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 4, 1, 1, 3, 1, 2, 1, 1, 1, 1, 3, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 3, 2, 3, 4, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 3, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 3, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 3, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 3, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 3, 2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 3, 1, 1, 2, 1, 1, 1, 2, 4, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 3, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 3, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 3, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 3, 1, 2, 1, 1, 1, 4, 3, 1, 1, 2, 2, 1, 3, 1, 1, 1, 1, 2, 3, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 3, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 3, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 4, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 3, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 3, 2, 3, 1, 1, 1, 1, 2, 3, 1, 3, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 3, 1, 2, 2, 2, 3, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 3, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 3, 1, 1, 1, 2, 3, 1, 1, 1, 3, 3, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 3, 1, 2, 2, 2, 3, 1, 3, 1, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 3, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2]

True

In [212]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 0.7823 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_16104\356962432.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache):
     4      1001       2489.0      2.5      0.0      SixCounter    = 0
     5      1001       2437.0      2.4      0.0      SixCounterMax = 0
     6      1001       2445.0      2.4      0.0      j = 0
     7     77342     190859.0      2.5      2.4      while N > 0:
     8                                                   # TODO: use continued division for this as well? Or can we just use the remainder instead of the number?
     9     76341    4213638.0     55.2     53.9          Digit = N % 10
    10     76341     188286.0      2.5      2.4          if Digit == 6:
    11      7925      21975.0      2.8      0.3              SixCounter   += 1
 

In [216]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 0.388465 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_16104\3149308276.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache):
     4      1001       2521.0      2.5      0.1      SixCounter    = 0
     5      1001       2312.0      2.3      0.1      SixCounterMax = 0
     6      1001       2215.0      2.2      0.1      j = 0
     7     77342     194824.0      2.5      5.0      while N > 0:
     8     76341     157660.0      2.1      4.1          try:
     9     76341     265909.0      3.5      6.8              Q, R = Cache[j]
    10     76264     315681.0      4.1      8.1              R    = (R << 1) | (N & 1)
    11     76264     196525.0      2.6      5.1              if R >= D:
    12     37555     124209.0      3.3      3.2                  R = R - D
    13    

## Improve Candidate selection

In [52]:
from line_profiler import LineProfiler

In [149]:
def count_sixes_old(N, D, Cache):
    SixCounter    = 0
    SixCounterMax = 0
    j = 0
    while N > 0:
        try:
            Q, R = Cache[j]
            R    = (R << 1) | (N & 1)
            if R >= D:
                R = R - D
                N = (Q << 1) | 1
            else:
                N = (Q << 1)
        except KeyError:
            R = N  % D
            N = N // D
        Cache[j] = N, R
        j += 1
        
        Digit = R % 10
        if Digit == 6:
            SixCounter   += 1
            SixCounterMax = max(SixCounterMax, SixCounter)
        else:
            SixCounter = 0
        
    return SixCounterMax

In [151]:
profiler = LineProfiler()
@profiler
def count_sixes(N, D, Cache, k=1):
    SixCounter    = 0
    SixCounterMax = 0
    j = 0
    while N > 0:
        try:
            Q, R = Cache[j]
            R    = (R << 1) | (N & 1)
            if R >= D:
                R = R - D
                N = (Q << 1) | 1
            else:
                N = (Q << 1)
        except KeyError:
            R = N  % D
            N = N // D
        Cache[j] = N, R
        
        Digit = R % 10
        if Digit == 6:
            SixCounter += 1
            if SixCounter == 5:
                # Go back and run a more expensive check to reduce likelyhood of selecting (wrong) candidates.
                # If this is a goliath number, then this needs to now be all sixes.
                R = (Cache[j-1][1]) // 10 # We already know the last digit is a six, so skip it.
                for _ in range(k):
                    Digit = R % 10
                    if Digit == 6:
                        R = R // 10
                    else:
                        SixCounter = 1 # Set to one because the digit here at j is actually a 6.
                        break
            SixCounterMax = max(SixCounterMax, SixCounter) # This is not in the else block to support k = 0
        else:
            SixCounter = 0
        j += 1
    return SixCounterMax

In [54]:
%%time
i        = 100_000
Divisor = 10**399

Building Cache...
1
Done!
CPU times: total: 15.6 ms
Wall time: 17.1 ms


In [66]:
G = int('666'*666)

In [153]:
%%time
i      = 1_400_052
Number = 1<<(i)
count_sixes(Number, Divisor, {}, k=2)

CPU times: total: 3.3 s
Wall time: 3.3 s


4

In [154]:
count_sixes_old(Number, Divisor, {})

5